# 509 Final Project

## Rsolve dependencies

In [1]:
! pip install newsapi-python

## Globally import libraries

In [2]:
import numpy as np
import pandas as pd
import pymysql as mysql
import matplotlib.pyplot as plt
import os
import shutil
import re
import logging
import time
import zipfile
import requests
from bs4 import BeautifulSoup
import datetime as dt
import re
import regex as rex
from collections import defaultdict, Counter
import random
#import mysql.connector

# Set pandas global options
pd.options.display.max_rows = 17

In [3]:
today = dt.date.today()
print(today)
print(type(today))

2023-06-08
<class 'datetime.date'>


## Connect to NewsAPI client

In [4]:
from newsapi import NewsApiClient

api_key = os.environ['NewsAPIKey']

# Init
newsapi = NewsApiClient(api_key=api_key)

## Pull article info from API

# /v2/top-headlines/sources
sources = newsapi.get_sources()
print(sources)

In [5]:
def news_api_urls(q=None,
                  s=None,
                  d_from='2023-05-01',
                  d_to='2023-05-31',
                  api_lst=[]):
    all_articles = newsapi.get_everything(q=q,
                                          sources=s,
                                          from_param=d_from,
                                          to=d_to,
                                          language='en',
                                          sort_by='relevancy',
                                          page=1)

    print(type(all_articles))
    print(all_articles)
    #print('Article list: ', all_articles['articles'])
    for article in all_articles['articles']:
        print('Source ID:', article['source']['id'])
        print('Source name:', article['source']['name'])
        print('Author:', article['author'])
        print('Title:', article['title'])
        print('URL:', article['url'])
        print('Publish date:', article['publishedAt'])
        print('Article text:', article['content'], '\n')

    # Create a list of tuples from the dictionary data
    source_data01 = [(a['source']['name'],
                      a['author'],
                      a['title'],
                      a['url'],
                      a['publishedAt'],
                      a['content'])
                     for a in all_articles['articles']]

    api_lst.extend(source_data01)
    #print(api_lst)
    print(len(api_lst))

## Connect to API to access URLs

### Set API filter parameters

In [6]:
# Citation comes from Allsides Media Bias Chart
#source_lst = ['cnn', 'reuters', 'fox-news']
#source_lst = ['cnn', 'newsweek', 'fox-news']
#source_lst = ['newsweek']
#source_lst = ['cnn', 'the-washington-post', 'fox-news', 'breitbart-news']
#source_lst = ['cnn', 'fox-news']
source_lst = ['breitbart-news']

#date_lst = ['2023-06-03', '2023-06-02']
date_lst = ['2023-06-03', '2023-06-02', '2023-06-01', '2023-05-31', '2023-05-30', '2023-05-29']
#date_lst = ['2023-06-01', '2023-05-31']
#date_lst = ['2023-05-30', '2023-05-29']
#date_lst = ['2023-05-17', '2023-05-16', '2023-05-15', '2023-05-14', '2023-05-13']
#date_lst = ['2023-05-12', '2023-05-11', '2023-05-10', '2023-05-09']
#date_lst = ['2023-05-08']
#date_lst = ['2023-05-07', '2023-05-06', '2023-05-05']

# Comes from academic text (fill in citation from DF)
q_word_lst = ['gender OR male OR female OR transgender', 'security AND (social OR national)',
              'justice OR surveillance', 'healthcare OR "health care"',
              '(political AND (bias OR party)) OR republican OR democrat OR election', '(policy AND (drug OR "affirmative action")) OR regulate OR regulation']

#q_word_lst = ['government']

### Access API

In [7]:
api_record_lst01 = []
for s in source_lst:
    print(f'Source: {s}')
    for d in date_lst:
        print(f'Date: {d}')
        for q in q_word_lst:
            print(f'Query word: {q}')
            time.sleep(5 + 11 * random.random())
            news_api_urls(q=q,
                          s=s,
                          d_from=d,
                          d_to=d,
                          api_lst=api_record_lst01)
            print(s, d, q)
    time.sleep(10 + 13 * random.random())

print(api_record_lst01)
print(len(api_record_lst01))

Source: breitbart-news
Date: 2023-06-03
Query word: gender OR male OR female OR transgender
<class 'dict'>
{'status': 'ok', 'totalResults': 7, 'articles': [{'source': {'id': 'breitbart-news', 'name': 'Breitbart News'}, 'author': 'Neil Munro, Neil Munro', 'title': "Poll: Public 'Strongly' Rejects Transgenderism Despite Corporate 'Pride'", 'description': 'A clear majority of Americans strongly oppose the transgender ideology despite a roar of pro-transgenderism rhetoric from the establishment.', 'url': 'https://www.breitbart.com/social-justice/2023/06/03/poll-public-strongly-rejects-transgenderism-despite-corporate-pride/', 'urlToImage': 'https://media.breitbart.com/media/2023/06/Target-Protest-640x335.jpg', 'publishedAt': '2023-06-03T17:13:49Z', 'content': 'Americans are uniting in broad opposition to the transgender ideology despite a roar of pro-transgenderism rhetoric from Fortune 500 corporations, government officials, and establishment media.\r\nFift… [+7740 chars]'}, {'source': {'

In [8]:
print(api_record_lst01)
print(len(api_record_lst01))

[('Breitbart News', 'Neil Munro, Neil Munro', "Poll: Public 'Strongly' Rejects Transgenderism Despite Corporate 'Pride'", 'https://www.breitbart.com/social-justice/2023/06/03/poll-public-strongly-rejects-transgenderism-despite-corporate-pride/', '2023-06-03T17:13:49Z', 'Americans are uniting in broad opposition to the transgender ideology despite a roar of pro-transgenderism rhetoric from Fortune 500 corporations, government officials, and establishment media.\r\nFift… [+7740 chars]'), ('Breitbart News', 'John Nolte, John Nolte', 'Nolte: Disney Groomers Face Another Potential Flop with Pixar’s ‘Elemental’', 'https://www.breitbart.com/entertainment/2023/06/03/nolte-disney-groomers-face-another-potential-flop-pixars-elemental/', '2023-06-03T16:05:12Z', 'The child exploiters at Disney are looking at another potential flop with Pixar’s poorly reviewed Elemental.\r\nHow’s your Pride Month going, groomers?\r\nThe perverts and fetishists at Disney reportedly… [+2566 chars]'), ('Breitbart News

In [9]:
#api_record_lst01 = [(1, 1, 1), (2, 2, 2), (3, 3, 3), (1, 1, 1), (4, 4, 4), (3, 3, 3)]
#print(api_record_lst01)
api_record_set01 = set(api_record_lst01)
print(api_record_set01)
api_record_lst02 = list(api_record_set01)
print(api_record_lst02)
print(len(api_record_lst02))

{('Breitbart News', 'David Ng, David Ng', "Looney Tunes Uses Bugs Bunny to Celebrate Drag Queens for Pride Month: 'Get Your Drag On'", 'https://www.breitbart.com/entertainment/2023/06/02/looney-tunes-uses-bugs-bunny-to-celebrate-drag-queens-for-pride-month-get-your-drag-on/', '2023-06-02T18:57:09Z', 'Looney Tunes — the classic children’s animated brand from Warner Bros. — is celebrating Pride Month by encouraging people to dress up as drag queens.\r\nIn a tweet Thursday, which was the first day of … [+1456 chars]'), ('Breitbart News', 'John Hayward', 'World Health Organization Grants North Korea Spot on Executive Board', 'https://www.breitbart.com/asia/2023/06/01/world-health-organization-grants-north-kore-spot-executive-board/', '2023-06-01T19:48:51Z', 'The World Health Organization (W.H.O.) concluded its annual meeting on Tuesday with the astonishing spectacle of electing North Korea among the worlds deadliest regimes, a psychopathic nuclear-armed … [+6240 chars]'), ('Breitbart News'

## Initiate MySQL connection

In [10]:
'''Set local environment variables to hide user name & password citation:
https://www.geeksforgeeks.org/how-to-hide-sensitive-credentials-using-python/'''

user_name = os.environ['MySQLUSRAC']
user_pass = os.environ['MySQLPWDAC']

# Instantiate connection
db_conn = mysql.connect(host='localhost',
                        port=int(3306),
                        user=user_name,
                        passwd=user_pass,
                        db='509_final_proj')

# Create a cursor object
cursor = db_conn.cursor()

In [11]:
tbl_names = pd.read_sql('SHOW TABLES', db_conn)

display(tbl_names)
print(type(tbl_names))

C:\Users\acarr\AppData\Local\Temp\ipykernel_3244\4193860975.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tbl_names = pd.read_sql('SHOW TABLES', db_conn)


,Tables_in_509_final_proj
0,nar_temp
1,news_articles


<class 'pandas.core.frame.DataFrame'>


### Establish logging policy

In [12]:
'''Logging citations (see additional code in following code blocks:
OpenAI. (2021). ChatGPT [Computer software]. https://openai.com/;
https://docs.python.org/3/howto/logging.html#logging-basic-example;
https://docs.python.org/3/howto/logging.html#logging-to-a-file;
https://docs.python.org/3/howto/logging-cookbook.html#using-a-rotating-log-file-handler;
https://docs.python.org/3/howto/logging-cookbook.html#using-a-timed-rotating-file-handler'''

# Set up logging
logging.basicConfig(level=logging.INFO,
                    filename='pymysql.log',
                    filemode='a',
                    format='>>>>>>>>>>>>>><<<<<<<<<<<<<<\n%(asctime)s - %(levelname)s - %(message)s')

### Update individual tables

#### Update `news_articles` table from API

In [13]:
nat_tbl_name = 'nar_temp'
nwa_tbl_name = 'news_articles'

In [14]:
'''Using cursor and loading into temp file:
OpenAI. (2021). ChatGPT [Computer software]. https://openai.com/;
https://pynative.com/python-mysql-insert-data-into-database-table/'''

# Execute query and measure execution time
start_time = time.time()

# Wipe temp table
try:
    nat_dlt_tble_stmnt = f"""DELETE FROM {nat_tbl_name}"""
    cursor.execute(nat_dlt_tble_stmnt)
    logging.info(f'Successfully executed query:\n{nat_dlt_tble_stmnt}\n\nRecords scanned: {cursor.rowcount}')
except mysql.Error as e:
    logging.error(f'Error executing query:\n{nat_dlt_tble_stmnt}\n\n{e}')
finally:
    end_time = time.time()
    logging.info(f'Time taken: {end_time - start_time:.3f} seconds\n>>>>>>>>>>>>>><<<<<<<<<<<<<<\n\n')

# Execute query and measure execution time
start_time = time.time()

# Load data from CSV file into a temporary table
try:
    nat_csv_load_stmnt = f"""
    INSERT INTO {nat_tbl_name}
    (
    source_name,
    author,
    title,
    url,
    publish_date,
    content
    )
    VALUES (%s, %s, %s, %s, %s, %s)
    """

    # Execute the query with multiple values
    cursor.executemany(nat_csv_load_stmnt, api_record_lst02)
    #cursor.execute(nat_csv_load_stmnt)
    logging.info(f'Successfully executed query:\n{nat_csv_load_stmnt}\n\nRecords scanned: {cursor.rowcount}')
except mysql.Error as e:
    logging.error(f'Error executing query:\n{nat_csv_load_stmnt}\n\n{e}')
finally:
    end_time = time.time()
    logging.info(f'Time taken: {end_time - start_time:.3f} seconds\n>>>>>>>>>>>>>><<<<<<<<<<<<<<\n\n')

# Execute query and measure execution time
start_time = time.time()

# Insert new records into main table
try:
    nwa_load_stmnt = f"""
    INSERT INTO {nwa_tbl_name}
    (
    source_name,
    author,
    title,
    url,
    publish_date,
    content
    )
    SELECT
        tp.source_name,
        tp.author,
        tp.title,
        tp.url,
        tp.publish_date,
        tp.content
    FROM {nat_tbl_name} AS tp
    LEFT JOIN {nwa_tbl_name} AS mn
        ON tp.title = mn.title
            AND CAST(LEFT(tp.publish_date, 10) AS DATE) = CAST(LEFT(mn.publish_date, 10) AS DATE)
            AND tp.author = mn.author
    """
    cursor.execute(nwa_load_stmnt)
    logging.info(f'Successfully executed query:\n{nwa_load_stmnt}\n\nRecords scanned: {cursor.rowcount}')
except mysql.Error as e:
    logging.error(f'Error executing query:\n{nwa_load_stmnt}\n\n{e}')
finally:
    end_time = time.time()
    logging.info(f'Time taken: {end_time - start_time:.3f} seconds\n>>>>>>>>>>>>>><<<<<<<<<<<<<<\n\n')

# Execute query and measure execution time
start_time = time.time()

# Wipe temp table
try:
    cursor.execute(nat_dlt_tble_stmnt)
    logging.info(f'Successfully executed query:\n{nat_dlt_tble_stmnt}\n\nRecords scanned: {cursor.rowcount}')
except mysql.Error as e:
    logging.error(f'Error executing query:\n{nat_dlt_tble_stmnt}\n\n{e}')
finally:
    end_time = time.time()
    logging.info(f'Time taken: {end_time - start_time:.3f} seconds\n>>>>>>>>>>>>>><<<<<<<<<<<<<<\n\n')

### Commit changes and close cursor and connection instances

In [15]:
# Commit the changes to the database
db_conn.commit()

# Close the cursor and database connection
cursor.close()
db_conn.close()